In [ ]:
import sys
print(sys.path)
sys.path.insert(0, sys.path[0].replace('/src/jupyter_nb', ''))
print(sys.path)
import src.utils as utils
import src.myenv as myenv
import pandas as pd

In [ ]:
from datetime import datetime, timedelta
import pytz

def get_start_timestamp_for_interval(interval):
	date = None
	match interval:
		case '1m':
			date = datetime.now(pytz.utc) - timedelta(days=30*9) # datetime(2023, 6, 1, 0, 0, 0, 0, pytz.utc)
		case '5m':
			date = datetime.now(pytz.utc) - timedelta(days=30*12) # datetime(2023, 1, 1, 0, 0, 0, 0, pytz.utc)
		case '15m':
			date = datetime.now(pytz.utc) - timedelta(days=365*2) # datetime(2022, 1, 1, 0, 0, 0, 0, pytz.utc)
		case '30m':
			date = datetime.now(pytz.utc) - timedelta(days=365*3) # datetime(2021, 1, 1, 0, 0, 0, 0, pytz.utc)
		case '1h':
			date = datetime.now(pytz.utc) - timedelta(days=365*6) # datetime(2017, 1, 1, 0, 0, 0, 0, pytz.utc)
	
	return int(date.timestamp() * 1000)
	

def reduce_database(interval_list = ['1m', '5m', '15m', '30m', '1h']):		
	for symbol in utils.get_symbol_list():
		for interval in interval_list:
			data_file = f'{myenv.datadir}/{symbol}/{symbol}_{interval}.dat'
			parsed_date = get_start_timestamp_for_interval(interval)
			print(data_file)
			data = utils.get_data(symbol=symbol, save_database=False, interval=interval, columns=myenv.all_klines_cols, parse_dates=False)
			data.info()
			data = data[(data['open_time'] >= parsed_date)]
			data.info()
			data.to_csv(data_file, sep=';', index=False, )
	

In [ ]:
reduce_database()

In [ ]:
data = utils.get_data(symbol='BTCUSDT', save_database=False, interval='1h', columns=myenv.all_klines_cols, parse_dates=True)
data.info()

In [ ]:
import numpy as np

min_date = data['open_time'].min()
max_date = data['open_time'].max()
print(f'Data Loaded: Min date: {min_date} - Max date: {max_date}')
validate_start_data = max_date - np.timedelta64(myenv.days_to_validate_train, 'D')

print(f'Filtering all data: days_to_validate_train_data: {myenv.days_to_validate_train} days')
min_index = data[data['open_time'] < validate_start_data].tail(myenv.rows_to_train).index.min()
real_rows_to_train = data[data['open_time'] < validate_start_data].tail(myenv.rows_to_train).shape[0]
real_rols_to_validate = data[data['open_time'] >= validate_start_data].shape[0]
data = data[data.index >= min_index]

min_date = data['open_time'].min()
max_date = data['open_time'].max()

print(f'All Data Filtered: train_start_date: {min_date} - validate_start_data: {validate_start_data} - max_date: {max_date}')
print(f'rows_to_train: {real_rows_to_train} - rows_to_validate: {real_rols_to_validate} - All Data Shape: {data.shape[0]}')

In [ ]:
from datetime import datetime, timedelta
print("Current time: ", datetime.now())
print(datetime.now() - timedelta(days=365))